# 전략 볼린저 밴드 이용 추세추종 기법 (심화버전 MFI)

매수 : 주가 상단 밴드 접근, 지표 강세를 확증할 때 매수  
%b > 0.8 and MFI > 80  
매도 : 주가 하단 밴드 접근, 지표 약세를 확증할 때 매도  
%b < 0.2 and MFI < 20   

#### test data 삼성전자

In [63]:
import time
from tqdm import tqdm
import numpy as np
from copy import deepcopy
from dateutil.parser import parse
import datetime as dt
from pykrx import stock
import sys
sys.path.append('D:/ML/Kis_Api_test/')
from BackTesting_System.backtesting import BackTesting,Utils

In [58]:
import pandas as pd 
df_raw = pd.read_csv('../../../mom.csv', index_col=0)

In [3]:
# df_raw = df_raw.drop(df_raw.iloc[:,8:],axis=1)

#### 전종목 볼린저밴드만들기

In [4]:
def make_bolinger_band(df):
    object_list = df['종목명'].unique()
    for i in tqdm(object_list):
        idx = df[df['종목명']==i].index
        df.loc[idx,'MA20'] = df.loc[idx,'종가'].rolling(window=20).mean()
        df.loc[idx,'stddev'] = df.loc[idx,'종가'].rolling(window=20).std()
    df['upper'] = df['MA20'] + (df['stddev'] * 2)   
    df['lower'] = df['MA20'] - (df['stddev'] * 2)
    df['PB'] = (df['종가'] - df['lower']) / (df['upper'] - df['lower'])
#     df['bandwidth'] = (df['upper'] - df['lower']) / df['MA20'] * 100
    return df


In [5]:
df_bol = make_bolinger_band(df_raw)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2436/2436 [05:48<00:00,  7.00it/s]


In [6]:
# df_bol.to_csv('bolinger.csv')

#### 반전매매 기법 일중강도

In [7]:
def make_intra_inten(df):
    df['II'] = (2*df['종가']-df['최고가']-df['최저가'])/(df['최고가']-df['최저가'])*df['누적거래량'] 
    object_list = df['종목명'].unique()
    for i in tqdm(object_list):
        idx = df[df['종목명']==i].index
        df.loc[idx,'IIP21']= df.loc[idx,'II'].rolling(window=21).sum()/df.loc[idx,'누적거래량'].rolling(window=21).sum()*100  # ②
    return df


In [8]:
df_result = make_intra_inten(df_bol)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2436/2436 [05:33<00:00,  7.30it/s]


In [9]:
def make_raking(df):
    df = deepcopy(df)
    for day, group_df in tqdm(df.groupby('날짜', sort=True)):
        idx = group_df.index
        df.loc[idx,'ranking'] = group_df['IIP21'].rank(method='min', ascending=False)
    return df

In [10]:
df_test = make_raking(df_result)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1213/1213 [00:01<00:00, 720.15it/s]


In [186]:
def make_signal_bol(df, rrr, risk):
    df = deepcopy(df)
    df['buy'] = ((df['PB'] < 0.05) & (df['IIP21'] > 0)) #& (df['ranking'] < 100)) #& (df['누적거래대금'] > 1000000000))
    df['sell'] = (df['PB'] > 0.95) & (df['IIP21'] < 0)
#     buf = sorted(df[df['buy'] == True].index)
#     for idx1, idx2 in tqdm(zip(buf[:-1], buf[1:])):
#         df.loc[idx1:idx2-1, '익절'] = df.loc[idx1, '종가'] + (df.loc[idx1, '종가'] * risk * rrr)  
#         df.loc[idx1:idx2-1, '손절'] = df.loc[idx1, '종가'] - (df.loc[idx1, '종가'] * risk)
#     df['sell_profit'] = (df['종가'] >= df['익절']) 
#     df['sell_risk'] =  (df['종가'] <= df['손절'])
    return df

In [187]:
df_signal = make_signal_bol(df_test, 2.23, 0.02)

In [188]:
df_signal['buy'].value_counts()

False    2581457
True       40177
Name: buy, dtype: int64

In [189]:
df_signal['sell'].value_counts()

False    2556388
True       65246
Name: sell, dtype: int64

In [190]:
code_kospi = pd.read_csv('../../SRC/current_kospi_codes.csv')
code_kosdaq = pd.read_csv('../../SRC/current_kosdaq_codes.csv')
code = pd.concat([code_kospi,code_kosdaq])
code.rename(columns={'회사명':'종목명'},inplace=True)
code['종목코드'] = code['종목코드'].astype('str')

df_signal = df_signal.merge(code,on='종목명',how='inner')
df_signal.rename(columns={'종목코드':'코드'},inplace=True)
df_signal['구매수량']=1

In [191]:
df_signal['코드'] = df_signal['코드'].apply(lambda x: str(x).zfill(6))

In [192]:
# df_signal = df_signal[df_signal['날짜'] > '2021-07-01']

In [193]:
df_signal['buy'].value_counts()

False    2506036
True       38859
Name: buy, dtype: int64

In [194]:
df_signal['sell'].value_counts()

False    2481546
True       63349
Name: sell, dtype: int64

In [195]:
bt = BackTesting()
ut = Utils()

In [198]:
first_date = parse(sorted(df_signal['날짜'].unique())[0])
last_date = parse(sorted(df_signal['날짜'].unique())[-1])
df_signal_test = df_signal
df_stock = stock.get_index_fundamental(str(first_date)[:10], str(last_date)[:10], '1001')
bt.MONEY = 10000000
first_monry = bt.MONEY
bt.buy_sell(df=df_signal_test,buy='buy',sell='sell')
bt.update_information()
print('-'*50)
print('기간:' + str(first_date) + '~' + str(last_date))
print('승률:', round(bt.ECONO_INFORM[bt.ECONO_INFORM['실현손익'] > 0.0].shape[0] / bt.ECONO_INFORM.shape[0]*100,2),'%')
print('승리횟수:' + str(bt.ECONO_INFORM[bt.ECONO_INFORM['실현손익'] > 0.0].shape[0]))
print('전체횟수:' + str(bt.ECONO_INFORM.shape[0]))
print('수익률', round(bt.ECONO_INFORM['실현손익'].sum()/first_monry * 100, 2),'%')
print('시장수익률:', + round(((df_stock.iloc[-1, 0] - df_stock.iloc[0, 0]) / df_stock.iloc[0, 0]) *100, 2), '%')
print()
print('-'*50)

--------------------------------------------------
기간:2018-01-29 00:00:00~2022-12-29 00:00:00
승률: 52.6 %
승리횟수:515
전체횟수:979
수익률 5.62 %
시장수익률: -13.92 %

--------------------------------------------------


In [197]:
res = []
for days in [365]:
    first_date = parse(sorted(df_signal['날짜'].unique())[0])
    last_date = parse(sorted(df_signal['날짜'].unique())[-1])
    term = dt.timedelta(days=days)
    term_bias_date = first_date
    while last_date - term_bias_date > term:        
        df_signal_test = df_signal[((df_signal['날짜'] > str(term_bias_date)) & (df_signal['날짜'] < str(term_bias_date + term)))]
        df_stock = stock.get_index_fundamental(str(term_bias_date)[:10], str(term_bias_date + term)[:10], '1001')
        bt.MONEY = 10000000
        first_monry = bt.MONEY
        bt.buy_sell(df=df_signal_test,buy='buy',sell='sell')
        bt.update_information()
        print('-'*50)
        print('기간:' + str(term_bias_date) + '~' + str(term_bias_date + term))
        print('승률:', round(bt.ECONO_INFORM[bt.ECONO_INFORM['실현손익'] > 0.0].shape[0] / bt.ECONO_INFORM.shape[0]*100,2),'%')
        print('승리횟수:' + str(bt.ECONO_INFORM[bt.ECONO_INFORM['실현손익'] > 0.0].shape[0]))
        print('전체횟수:' + str(bt.ECONO_INFORM.shape[0]))
        print('수익률', round(bt.ECONO_INFORM['실현손익'].sum()/first_monry * 100, 2),'%')
        print('시장수익률:', + round(((df_stock.iloc[-1, 0] - df_stock.iloc[0, 0]) / df_stock.iloc[0, 0]) *100, 2), '%')
        print()
        print('-'*50)
        term_bias_date = term_bias_date + term
        res.append(round(bt.ECONO_INFORM[bt.ECONO_INFORM['실현손익'] > 0.0].shape[0] / bt.ECONO_INFORM.shape[0]*100,2))
    
    df_signal_test = df_signal[((df_signal['날짜'] > str(term_bias_date)) & (df_signal['날짜'] <= str(last_date)))]
    df_stock = stock.get_index_fundamental(str(term_bias_date)[:10], str(last_date), '1001')
    bt.MONEY = 10000000
    first_monry = bt.MONEY
    bt.buy_sell(df=df_signal_test,buy='buy',sell='sell')
    bt.update_information()
    print('-'*50)
    print('기간:' + str(term_bias_date) + '~' + str(last_date))
    print('승률:', round(bt.ECONO_INFORM[bt.ECONO_INFORM['실현손익'] > 0.0].shape[0] / bt.ECONO_INFORM.shape[0]*100,2),'%')
    print('승리횟수:' + str(bt.ECONO_INFORM[bt.ECONO_INFORM['실현손익'] > 0.0].shape[0]))
    print('전체횟수:' + str(bt.ECONO_INFORM.shape[0]))
    print('수익률', round(bt.ECONO_INFORM['실현손익'].sum()/first_monry * 100, 2),'%')
    print('시장수익률:', + round(((df_stock.iloc[-1, 0] - df_stock.iloc[0, 0]) / df_stock.iloc[0, 0]) *100, 2), '%')
    print()
    print('-'*50)
    
    res.append(round(bt.ECONO_INFORM[bt.ECONO_INFORM['실현손익'] > 0.0].shape[0] / bt.ECONO_INFORM.shape[0]*100,2))

--------------------------------------------------
기간:2018-01-29 00:00:00~2019-01-29 00:00:00
승률: 46.51 %
승리횟수:80
전체횟수:172
수익률 -7.04 %
시장수익률: -15.97 %

--------------------------------------------------
--------------------------------------------------
기간:2019-01-29 00:00:00~2020-01-29 00:00:00
승률: 55.35 %
승리횟수:119
전체횟수:215
수익률 -2.05 %
시장수익률: 0.09 %

--------------------------------------------------
--------------------------------------------------
기간:2020-01-29 00:00:00~2021-01-28 00:00:00
승률: 65.93 %
승리횟수:149
전체횟수:226
수익률 13.54 %
시장수익률: 40.44 %

--------------------------------------------------
--------------------------------------------------
기간:2021-01-28 00:00:00~2022-01-28 00:00:00
승률: 66.67 %
승리횟수:148
전체횟수:222
수익률 13.08 %
시장수익률: -13.22 %

--------------------------------------------------
--------------------------------------------------
기간:2022-01-28 00:00:00~2022-12-29 00:00:00
승률: 34.38 %
승리횟수:55
전체횟수:160
수익률 0.09 %
시장수익률: -16.03 %

-------------------------------------

In [184]:
def risk_reward_raio(ex, wr):
    return round(((1 - wr) + ex)/ wr, 2)

In [173]:
bt.JOUNAL

,날짜,종목코드,체결단가,체결수량,매매비용,매매구분
0,2021-12-10,151910,5810,1,8.0,매수
1,2021-12-13,151910,5760,1,8.0,매수
2,2021-12-14,091090,1015,1,1.0,매수
3,2021-12-14,220180,3600,1,5.0,매수
4,2021-12-14,054210,10600,1,15.0,매수
...,...,...,...,...,...,...
971,2022-12-06,072520,2385,1,3.0,매도
972,2022-12-07,366330,2030,1,3.0,매도
973,2022-12-08,088800,5740,1,8.0,매도
974,2022-12-09,340570,36300,1,51.0,매도


#### BackTest

In [21]:
import sys
sys.path.append('D:/ML/Kis_Api_test/')
from BackTesting_System.backtesting import BackTesting, Utils

In [22]:
bt = BackTesting()
ut = Utils()
bt

In [23]:
bt.MONEY = 10000000
first_monry = bt.MONEY

In [24]:
bt.buy_sell(df=df_signal,buy='buy',sell='sell')

In [25]:
bt.update_information()

In [26]:
bt.JOUNAL

,날짜,종목코드,체결단가,체결수량,매매비용,매매구분
0,2021-07-02,088980,12400,3,52.0,매수
1,2021-07-05,088980,12300,3,52.0,매수
2,2021-07-05,066360,1945,3,8.0,매수
3,2021-07-06,088980,12350,3,52.0,매수
4,2021-07-09,039570,13600,3,57.0,매수
...,...,...,...,...,...,...
284,2022-12-23,002700,1865,3,8.0,매수
285,2022-12-26,002700,1845,3,8.0,매수
286,2022-12-28,175330,7910,3,33.0,매수
287,2022-12-28,003540,13250,3,56.0,매수


In [27]:
pd.options.display.max_rows = 10

In [28]:
print('시장 수익률: -30.17%')

시장 수익률: -30.17%


In [29]:
print('수익률',bt.ECONO_INFORM['실현손익'].sum()/first_monry * 100,'%')

수익률 -5.12385 %


In [30]:
print('승률:', round(bt.ECONO_INFORM[bt.ECONO_INFORM['실현손익'] > 0.0].shape[0] / bt.ECONO_INFORM.shape[0]*100,2),'%')

승률: 17.57 %


In [78]:
code_rand  = np.random.choice(bt.JOUNAL['종목코드'].unique())

In [79]:
bt.JOUNAL[bt.JOUNAL['종목코드'] == code_rand]

,날짜,종목코드,체결단가,체결수량,매매비용,매매구분
214,2022-06-20,327260,11050,3,47.0,매수
273,2022-11-08,327260,12000,3,51.0,매도


In [80]:
ut.making_mt_plot(df_signal, code=code_rand, jounal=bt.JOUNAL)

In [54]:
#매수수량의 문제를 해결해보자

In [55]:
df_signal['구매수량']=3

In [56]:
df_signal

,날짜,종가,시가,최고가,최저가,누적거래량,누적거래대금,종목명,5_mom,10_mom,...,상장일,MA20,stddev,upper,lower,PB,bandwidth,II,IIP21,체결수량
0,2022-12-22,10650,9980,11700,9930,28856232,309808795730,바이오노트,NaN,NaN,...,2022-12-22,NaN,NaN,NaN,NaN,NaN,NaN,-5.379975e+06,NaN,1
1,2022-12-23,10150,10600,11200,10100,8180753,86595242100,바이오노트,NaN,NaN,...,2022-12-22,NaN,NaN,NaN,NaN,NaN,NaN,-7.437048e+06,NaN,1
2,2022-12-26,9720,10550,10700,9650,4479667,45206813740,바이오노트,NaN,NaN,...,2022-12-22,NaN,NaN,NaN,NaN,NaN,NaN,-3.882378e+06,NaN,1
3,2022-12-27,9360,9770,9880,9130,4101930,38365054640,바이오노트,NaN,NaN,...,2022-12-22,NaN,NaN,NaN,NaN,NaN,NaN,-1.586080e+06,NaN,1
4,2022-12-28,8870,9390,9420,8700,3177379,28837782380,바이오노트,NaN,NaN,...,2022-12-22,NaN,NaN,NaN,NaN,NaN,NaN,-1.676950e+06,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
882122,2022-12-23,3795,3885,3930,3755,50452,191959725,한진중공업홀딩스,96.687898,95.591940,...,1956-03-03,3986.00,98.803367,4183.606733,3788.393267,0.016717,9.915039,-2.738823e+04,-30.467415,1
882123,2022-12-26,3785,3795,3800,3650,75667,282358160,한진중공업홀딩스,96.188056,94.862155,...,1956-03-03,3976.25,108.565709,4193.381418,3759.118582,0.059599,10.921417,6.053360e+04,-26.903812,1
882124,2022-12-27,3810,3750,3840,3750,30791,116935765,한진중공업홀딩스,97.442455,95.250000,...,1956-03-03,3962.50,111.443873,4185.387746,3739.612254,0.157900,11.249855,1.026367e+04,-21.251416,1
882125,2022-12-28,3890,3800,3930,3770,14512,55566740,한진중공업홀딩스,101.302083,96.526055,...,1956-03-03,3947.50,98.668771,4144.837541,3750.162459,0.354311,9.998102,7.256000e+03,-7.186440,1


In [47]:
bt1 = BackTesting()
ut = Utils()
bt1.MONEY = 100000000
bt1.buy_sell(df=df_signal,buy='buy',sell='sell')
bt1.update_information()
bt1.JOUNAL
bt1.ECONO_INFORM

,날짜,매수총합,매도총합,매매비용,실현손익
0,2018-02-28,4444266,0.0,6245.0,0.0
1,2018-03-02,5473362,0.0,7688.0,0.0
2,2018-03-05,8191392,0.0,11512.0,0.0
3,2018-03-06,5151711,0.0,7241.0,0.0
4,2018-03-07,6136677,0.0,8622.0,0.0
...,...,...,...,...,...
343,2022-08-05,0.0,44100,62.0,-19950
344,2022-08-08,26280,1983,39.0,-42
345,2022-09-20,0.0,7035,10.0,390
346,2022-11-16,26535,0.0,38.0,0.0


In [50]:
bt1.ECONO_INFORM['실현손익'].sum()/100000000*100

0.108543